In [1]:
import pandas as pd 
import numpy as np 
from bs4 import BeautifulSoup
from selenium import webdriver
import requests

In [5]:
pip install uszipcode


Note: you may need to restart the kernel to use updated packages.


In [2]:
from uszipcode import SearchEngine

search = SearchEngine(simple_zipcode=True)

In [3]:
#main function that will take the epicenter for disaster, calculate area of affect, find all zipcodes in that area
#and calculate a projected damage cost based on median home value and number of homes in area. with a penalty reduction
#assuming not all homes were destroyed.
def main_search(Zip, disaster_type):
    try:
    #setting dictionary for area covered by disaster, in KM   
        disaster_dist = {
            'hurricane': 100,
            'flood': 50,
            'fire':.5
        }
    #setting the effect ratio based on research from last 5 disasters by type, 
    #(More info needed for more accurate calculations)
        disaster_effect = {
            'hurricane': .1,
            'flood' : .81,
            'fire': 1
        }
        #instantiate the search engine
        search = SearchEngine(simple_zipcode=True)
        #searching for epicenter zipcode
        zipcode = search.by_zipcode(str(Zip))
        #Setting result as a list with every zipcode within a certain range. 
        result = search.by_coordinates(zipcode.lat,zipcode.lng, radius= (disaster_dist[disaster_type]), returns=1000)
        #printing the median household value for epicenter zipcode 
        print(f'The median house value of {Zip} ({zipcode.major_city}) is: {zipcode.median_home_value}')
        #iterate through our results and getting median home value times number of housing units in each zipcode effected by hurricane 
        value = 0
        for i in result:

            try: 
            #Try and except statements to account for null values where no houses exist.
                value += i.median_home_value * i.housing_units
            except:
                pass
        #Printing the total damage done minus the penalty for the specific disaster. 
        total_value = value * disaster_effect[disaster_type]
        function_total(total_value), print(f'for a {disaster_type}')
        
    except:
        print('Zipcode was not found/ was invalid')

In [4]:
main_search('21201', 'flood')

The median house value of 21201 (Baltimore) is: 208500
Zipcode was not found/ was invalid


In [83]:
def function_total(total_damage):
    convert = int(total_damage)
    formater = format(convert, ',')
    
    if len(formater.split(',')) == 3:
        print(f'Estimated damage between {int(formater[0]) * .75} and {formater[0]} Million dollars')
    elif len(formater.split(',')) == 4:
        print(f'Estimated damage between {int(formater[0]) * .75} and {formater[0]} Billion dollars')  
    

In [84]:
function_total(1000000000)

Estimated damage between 0.75 and 1 Billion dollars


In [74]:
samp2

'61,906,679,200'

In [80]:
if len(samp2.split(',')) == 3:
    print(f'Estimated damage between {int(samp2[0]) * .75} and {samp2[0]} Million dollars')
elif len(samp2.split(',')) == 4:
    print(f'Estimated damage between {int(samp2[0]) * .75} and {samp2[0]} Billion dollars')  

Estimated damage between 4.5 and 6 Billion dollars


In [32]:
all_zip = {}

In [33]:
new_zip = search.by_zipcode(10001)

In [34]:
new_zip.area_code_list

['718', '917', '347', '646']

In [35]:
for i in range( 10000,99999):
    new_zip = search.by_zipcode(str(i))
    if new_zip.population != None:
        all_zip[i] = {'latt': new_zip.lat,
                      'long': new_zip.lng,
                      'median_house_value': new_zip.median_home_value,
                      'housing_units': new_zip.housing_units,
                      'population' : new_zip.population}

In [40]:
pd.DataFrame(all_zip).T

,housing_units,latt,long,median_house_value,population
10001,12476.0,40.750,-73.990,650200.0,21102.0
10002,34541.0,40.720,-73.990,535600.0,81410.0
10003,31078.0,40.730,-73.990,817700.0,56024.0
10004,2197.0,40.700,-74.020,894200.0,3089.0
10005,5317.0,40.705,-74.005,1000001.0,7135.0
10006,1991.0,40.708,-74.013,714300.0,3011.0
10007,3143.0,40.714,-74.008,1000001.0,6988.0
10009,31709.0,40.730,-73.980,672800.0,61347.0
10010,18030.0,40.730,-73.980,746200.0,31834.0
10011,33252.0,40.740,-74.000,914500.0,50984.0


In [30]:
all_zip.shape

(5, 5)

In [31]:
all_zip.head()

,housing_units,latt,long,median_house_value,population
housing_units,NaN,NaN,NaN,NaN,NaN
latt,NaN,NaN,NaN,NaN,NaN
long,NaN,NaN,NaN,NaN,NaN
median_house_value,NaN,NaN,NaN,NaN,NaN
population,NaN,NaN,NaN,NaN,NaN


In [28]:
all_zip.set_index(keys=all_zip.columns)

ValueError: Length mismatch: Expected axis has 5 elements, new values have 30562 elements